In [1]:
import os
import sys
from glob import glob

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import desispec
from desispec.io import read_spectra, write_spectra
from desispec.spectra import Spectra

from desiutil.log import get_logger, DEBUG

from desidiff.src.group_tiles import *
from desidiff.src.dates_to_process import *
from desidiff.src.coadd import *
from desidiff.src.scores import *
from desidiff.src.ContinuumFitFilter_desidiff import *

from timedomain.sp_utils import SkyPortal as sp
import requests
import datetime
import heapq
import time

import numpy
from astropy.time import Time
from astropy.table import Table, vstack, unique, SortedArray

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from IPython import display

In [2]:
# read in and store in one place all the fibermap information in the spectra files
night = 20210608

### this needs to something other than cumulative?
for filename in glob(f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/*/{night}/spectra-*.fits"):
    print(filename)
    
    tile = filename.split('/')[-3]
    petal = filename.split('-')[1]
    current_spectra = read_spectra(filename)
    
    ### select only spectra from that night
    current_spectra = desispec.coaddition.coadd_cameras(current_spectra.select(nights = night))
    print(current_spectra.flux.keys())
    """    
    t = Table.read(filename, format='fits',hdu=1, memmap=True) 
    ##### DAVE'S ADDITION ##############
    targetcols = [i for i in t.colnames if i[-7:] =='_TARGET']
    nonzerocheck = [True in k for k in [[j != 0 for j in t[targetcols][i]] for i in range(len(t))]]
    #a really ugly line, basically generates a list of bools, 
    #True if there is at least one nonzero element in all columns ending in _TARGET
    t.remove_rows([i for i, val in enumerate(nonzerocheck) if not val])
    #This gets the index of all False values from the previous list and removes those rows
    t = t['TARGETID','TARGET_RA','TARGET_DEC','TILEID','OBJTYPE','PETAL_LOC','FIBERSTATUS','NIGHT']
    ######## END DAVE'S ADDITION ############
    t=t[numpy.logical_and(t['OBJTYPE']=='TGT', t['FIBERSTATUS']==0)]
    """

    for filename in glob(f"/global/cfs/cdirs/desi/spectro/redux/*/tiles/cumulative/{tile}/*/coadd-{petal}-{tile}*.fits"):
        print(filename)
        try:
            if filename.split('/')[-6] == 'fuji' or filename.split('/')[-6] == 'guadalupe':
                ref_spectra = desispec.coaddition.coadd_cameras(read_spectra(filename))
                print('ref_spectra from {}'.format(filename))
                norm = normalization(current_spectra.flux, current_spectra.mask, ref_spectra.flux, ref_spectra.mask)
                
                # need to instantiate a Spectra object for the difference. 
                # Using 'dif_spectra = Spectra()' is bugging on dif_spectra.mask type=NoneType, can't assign.
                dif_spectra = current_spectra
                # any problem with hardcoding in 'brz' for key in the following:
                for key in (current_spectra.flux).keys():
                    current_spectra.flux[key] = current_spectra.flux[key]/norm
                    current_spectra.ivar[key] = current_spectra.ivar[key]*norm**2  
                    dif_spectra.flux[key] = current_spectra.flux[key] - ref_spectra.flux[key]
                    # subtraction of current and reference fluxes
                    dif_spectra.flux[key] = current_spectra.flux[key] - ref_spectra.flux[key]
                    ### couldn't dif_spectra.mask[key] == 2 by summing current.mask and reference.mask
                    # summation of current and reference masks
                    dif_spectra.mask[key] = current_spectra.mask[key] + ref_spectra.mask[key]
                    # inverted summation of current and spectra inverse variance
                    ### still returning RuntimeWarning: divide by zero encountered in true_divide but not in infinite loop for the moment
                    dif_spectra.ivar[key] = 1./(1./current_spectra.ivar[key] + 1./ref_spectra.ivar[key])
                
                # mean-subtracted difference
                diff_flux_clipmean = clipmean(diff_flux,diff_ivar,diff_mask)
                
        except:
            pass    

/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/1097/20210608/spectra-6-1097-thru20210608.fits
INFO:spectra.py:291:read_spectra: iotime 0.724 sec to read spectra-6-1097-thru20210608.fits at 2022-06-22T14:01:48.717133
dict_keys(['brz'])
/global/cfs/cdirs/desi/spectro/redux/everest/tiles/cumulative/1097/20210608/coadd-6-1097-thru20210608.fits
/global/cfs/cdirs/desi/spectro/redux/daily/tiles/cumulative/1097/20210608/coadd-6-1097-thru20210608.fits
/global/cfs/cdirs/desi/spectro/redux/guadalupe/tiles/cumulative/1097/20210608/coadd-6-1097-thru20210608.fits
INFO:spectra.py:291:read_spectra: iotime 0.613 sec to read coadd-6-1097-thru20210608.fits at 2022-06-22T14:01:57.375698
ref_spectra from /global/cfs/cdirs/desi/spectro/redux/guadalupe/tiles/cumulative/1097/20210608/coadd-6-1097-thru20210608.fits
diff_flux done
/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/1097/20210608/spectra-4-1097-thru20210608.fits
INFO:spectra.py:291:read_spectra: iotime 0.

KeyboardInterrupt: 